### 무신사 크롤링하기

In [2]:
# 필요 라이브러리 가져오기
from bs4 import BeautifulSoup
import pandas as pd
from selenium import webdriver
import requests

In [4]:
driver = webdriver.Chrome()
driver.get('https://www.musinsa.com/categories/item/001005')

In [6]:
html = driver.page_source
soup = BeautifulSoup(html,'html.parser')

In [14]:
# 타이틀을 가지고 있는 클래스
pro_title = soup.select('.img-block')


##타이틀
# [i]['title']

In [15]:
pro_title_list = []
for title in pro_title:
    pro_title_list.append(title['title'])


In [17]:
len(pro_title_list)

90

In [24]:
## 브랜드
pro_brand = soup.select('.item_title > a')

## 정확한 브랜드 위치
# [i].text

In [25]:
pro_brnad_list = []
for brand in pro_brand:
    pro_brnad_list.append(brand.text)

In [27]:
len(pro_brnad_list)

90